In [1]:
import pickle
import numpy as np

from tools import kobak_tsne, PCA_sklearn, pearson_residuals, prepare_largest_batch, run_glmpca, sqrt_lazy

# Compute normalization

#### Load datasets

In [2]:
files =['datasets/retina/macosko_all/GSE63472_P14Retina_merged_digital_expression.txt_preprocessed.pickle',
        'datasets/retina/shekhar_bipolar/GSE81904_BipolarUMICounts_Cell2016.txt.gz_preprocessed.pickle',
        'datasets/retina/tran_ganglion/GSE133382_AtlasRGCs_CountMatrix.csv_preprocessed.pickle']

labels = ['macosko',
          'shekhar_bipolar',
          'tran_ganglion']

datasets=[]
for file,label in zip(files,labels):
    with open(file, "rb") as f:
        dataset = pickle.load(f)
        dataset['labelshort'] = label
    
    
    if label in ['macosko','shekhar_bipolar']:
        prepare_largest_batch(dataset,extra_keys=['replicates'])
    else:
        prepare_largest_batch(dataset)
    
    n_clusters = len(np.unique(dataset['clusters']))
    dataset['gene_means'] = np.array(np.mean(dataset['counts'],axis=0)).flatten()
    dataset['depths'] = np.array(np.sum(dataset['counts'],axis=1)).flatten()
    
    datasets.append(dataset)

Of 19252 total genes, returning 17973 genes that are detected in 5 or more cells.
Output shape: (24769, 17973)
Of 18149 total genes, returning 16520 genes that are detected in 5 or more cells.
Output shape: (13987, 16520)
Of 18137 total genes, returning 17685 genes that are detected in 5 or more cells.
Output shape: (15750, 17685)


### Analytical Pearson residuals

In [3]:
%%time
for dataset in datasets:
    dataset['residuals_theta100'] = pearson_residuals(dataset['counts'].toarray(),100)

CPU times: user 20.5 s, sys: 4.32 s, total: 24.8 s
Wall time: 24.8 s


### Sqrt transform

In [4]:
%%time
for dataset in datasets:
    dataset['sqrt_lazy'] = sqrt_lazy(dataset['counts'].toarray())

CPU times: user 6.82 s, sys: 1.07 s, total: 7.89 s
Wall time: 7.89 s


### GLM PCA

In [5]:
for dataset in datasets:
    run_glmpca(dataset['counts'].toarray(),fam='nb',optimize_nb_theta=False,theta=100,dataset_label=dataset['labelshort'])

In [6]:
for dataset in datasets:
    with open('glmpca_results/%sglmpca-py_nb_fixedTheta100_penalty1.pickle' % (dataset['labelshort']),"rb") as f:
        dataset['glmpca-py_nb_thetaFixed100'] = pickle.load(f)['factors']

# Compute tSNEs

In [7]:
%%time
for dataset in datasets:

    _,sqrt_lazy_init = PCA_sklearn(dataset['sqrt_lazy'],50,42)
    _ = kobak_tsne(dataset['sqrt_lazy'],name=dataset['labelshort']+'_sqrt_lazy_initsqrtLazy',do_pca=True,init=sqrt_lazy_init)
    _ = kobak_tsne(dataset['residuals_theta100'],name=dataset['labelshort']+'_residuals_theta100_initsqrtLazy',do_pca=True,init=sqrt_lazy_init)
    _ = kobak_tsne(dataset['glmpca-py_nb_thetaFixed100'],name=dataset['labelshort']+'_glmpca-py_nb_thetaFixed100_initsqrtLazy',do_pca=False,init=sqrt_lazy_init)

In [8]:
%load_ext watermark

In [9]:
watermark

Last updated: 2021-07-31T19:14:03.340668+02:00

Python implementation: CPython
Python version       : 3.8.0
IPython version      : 7.21.0

Compiler    : GCC 8.3.0
OS          : Linux
Release     : 3.10.0-957.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [10]:
watermark --iversions

numpy: 1.20.1

